# Edge Detection

We will now look at the problem of edge detection. We can define an edge as a rapid change in image intensity within a small region. We will look at edge detection with OpenCV. We will examine 3 functions:
* Sobel edge detection
* Laplaican edge detection
* Canny Edge detection

## Sobel Edge Detection

The Sobel Edge detection algorithm takes advantage of the fact that an edge is a rapid change in intensity. We know from calculus that the first derivative of a function gives us the instantenious change of a function at that point. Hence, the Sobel operator works by getting the first derivative of the image - or more exactly yet - the gradient (a pair of values), is found as the partial derivative w.r.t `x` and the partial derivitive w.r.t `y`.

These two values of the gradient can also be represented as a convolution with two kernels, the Sobel X kernel to approximate the X part of the gradient and the Y kernel to approximate the Y part.
<br><br>
* X-direction Kernel : $$\begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix}$$

* Y-direction Kernel : $$\begin{bmatrix} 1 & 2 & 1 \\ 0 & 0 & 0 \\ -1 & -2 & -1 \end{bmatrix}$$

The result of convolving an image with both kernels is the 'Sobel Edge Map'.

Let us try with an example image. We will convolve with the X-direction kernel to get the vertical edges and with the Y-direction kernel to get the horizontal edges.

### Convolving with the X-kernel

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
checker = cv2.imread('images/checkerboard_color.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
plt.figure(figsize=[3, 3])
plt.imshow(checker, cmap='gray')

We want to detect edges in the X-direction using the X-kernel.

In [ ]:
xkernel = np.array([[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]])
sobel_xfiltered = cv2.filter2D(checker, cv2.CV_64F, xkernel)
plt.imshow(sobel_xfiltered, cmap='gray')

### Convolving with the Y-kernel

We will now use the Y-kernel to detect horizontal edges. As with the X-kernel above, you notice that we use `cv2.CV_64F` for depth so as to capture all the ranges of values produced in the convolution operation. A depth allowing more values than 256 allows us more options in choosing the threshold values.

In [ ]:
# the y-kernel is similar to the x-kernel rotated by 90
ykernel = np.rot90(xkernel)
sobel_yfiltered = cv2.filter2D(checker, cv2.CV_64F, ykernel)
plt.imshow(sobel_yfiltered, cmap='gray')

Getting the complete Sobel edge map is similar to getting the magnitude of each edge. We can get an approximate magnitude by simply adding the X and Y maps together.

In [ ]:
sobel_filtered = sobel_xfiltered + sobel_yfiltered
plt.imshow(sobel_filtered, cmap='gray')

In [ ]:
_, sobel_thresh = cv2.threshold(sobel_filtered, -1, np.max(sobel_filtered), cv2.THRESH_BINARY_INV)
plt.imshow(sobel_thresh, cmap='gray')

We can create the same filter using OpenCV's `Sobel` method. We select the X or Y derivitive by setting the value of `dx` and `dy`.

In [ ]:
sobelx = cv2.Sobel(checker, cv2.CV_64F, 1, 0, ksize=3) #dx, dy (1, 0)
sobely = cv2.Sobel(checker, cv2.CV_64F, 0, 1, ksize=3) #dx, dy (0, 1)
plt.figure(figsize=[15, 3])
plt.subplot(121); plt.imshow(sobelx, cmap='gray'); plt.title('X - kernel')
plt.subplot(122); plt.imshow(sobely, cmap='gray'); plt.title('Y - kernel')

Getting the magnitude.

In [ ]:
sobel_full = np.sqrt(sobelx**2 + sobely**2)
plt.imshow(sobel_full, cmap='gray')

In [ ]:
_, sobel_thresh = cv2.threshold(sobel_full, 1, np.max(sobel_full), cv2.THRESH_BINARY)
plt.imshow(sobel_thresh, cmap='gray')

## Is the Sobel Kernel correlated or convoluted?

Convolution differs from correlation in that in correlation, the kernel is flipped twice, (along the Y-axis then the X-axis). This is no problem if the kernel is symmetrical along the diagonal. However, it becomes a problem if it is not. Let us try it out with the Sobel kernel.

In [ ]:
xpadded = np.pad(xkernel, (398, 399))

In [ ]:
xpadded_fft_shift = np.fft.fftshift(np.fft.fft2(xpadded))
checker_fft_shift = np.fft.fftshift(np.fft.fft2(checker))
plt.figure(figsize=[8, 4])
plt.subplot(121); plt.imshow(np.abs(xpadded_fft_shift), cmap='gray'); #plt.title('X - kernel')
plt.subplot(122); plt.imshow(np.abs(np.log(checker_fft_shift) + 1), cmap='gray'); #plt.title('Y - kernel #np,log is used for visibility

In [ ]:
sobel_edge_fft = xpadded_fft_shift * checker_fft_shift
plt.figure(figsize=[4, 4])
plt.imshow(np.abs(np.log(sobel_edge_fft + (1/np.abs(np.max(sobel_edge_fft))**51))), cmap='gray') #to remove log(0) but also avoid much interference.

In [ ]:
sobel_edge_ifft = np.fft.ifft2(np.fft.ifftshift(sobel_edge_fft))
plt.imshow(sobel_edge_ifft.real, cmap='gray')

In [ ]:
np.unique(sobel_edge_ifft.real)

### Convolving both kernels

In [ ]:
ypadded = np.pad(ykernel, (398, 399))
ypadded_fft_shift = np.fft.fftshift(np.fft.fft2(ypadded))
one_sobel_kernel = ypadded_fft_shift * xpadded_fft_shift
plt.imshow(np.abs(one_sobel_kernel), cmap='gray');

In [ ]:
one_sobel_map = one_sobel_kernel * checker_fft_shift
final_sobel = np.fft.ifft2(np.fft.ifftshift(one_sobel_map))

#standardize
#final_sobel = final_sobel + np.abs(np.min(final_sobel))
plt.figure(figsize=[10, 5])
plt.subplot(121); plt.imshow(np.abs(np.log(one_sobel_map + (1/np.abs(np.max(one_sobel_map))**51))), cmap='gray');
plt.subplot(122); plt.imshow(np.abs(final_sobel), cmap='gray');

In [ ]:
final_sobel.real[199, 499]

## Laplacian Operator

We have seen that the Sobel operator can be used to find edges by detecting rapid changes in intensity either in the X or Y direction. We have also seen that the operator produces two values: for X and Y and gives us the magnitude and location of an edge. The local maxima is what enables us to know whether a point defines an edge or not.

There  is also another operator known as the Laplacian Operator. The Laplacian Operator uses the second derivitive to detect edges. At the point of the local maxima in the first derivative, the is no change making the value 0 at the second derivative, but then there is a rapid decline of values immediately after. Hence, the edge is identified at the point of zero-crossing where the value quickly changes to negative as it crosses the local maxima in the first derivative.

Unlike the Sobel operator, the Laplacian operator produces one value since it is found by adding the second derivative w.r.t the X-direction and the second derivative w.r.t the Y-direction, hence resulting in one value.

### An Example

We can use OpenCV's `Laplacian` method to find the Laplacian edges.

In [ ]:
icon = cv2.imread('images/red_hibiscus.jpg', cv2.IMREAD_GRAYSCALE)
plt.imshow(icon, cmap='gray')

In [ ]:
icon_laplace = cv2.Laplacian(icon, cv2.CV_64F, ksize=3)

In [ ]:
plt.imshow(icon_laplace, cmap='gray')

Although difficult to see, the result of the Laplacian clearly marks out the flower's edges. To see it better, we can choose a ROI. From it we can clearly see the flower image.

In [ ]:
plt.imshow(icon_laplace[420:1000, 200:850], cmap='gray')

Next, we threshold the image to separate edges from non-edges. The threshold value represents the zero-crossings from the second derivative.

In [ ]:
_, icon_laplace_thresh = cv2.threshold(icon_laplace, 35, np.max(icon_laplace), cv2.THRESH_BINARY)
plt.imshow(icon_laplace_thresh, cmap='gray')

We have seen two edge-detection algorithms that rely on gradients to find the edges. The advantage of the Sobel operator is that it gives us both the magnitude of an edge and the direction. However, it requires two convolution operations. On the other hand, the Laplacian method requires only one convolution operation although it only tells us that there exists an edge at a certain point, not the magnitude or the direction of the edge. There is a method that combines both the above, taking the advantages of both, which we will look at in the next chapter.

## Reducing Noise

Noise affects the number and quality of edges we detect. Noise in an image can amplify non-edges to be recognized as edges. We can reduce noise by using Gaussian smoothing. As you know, a Gaussian is a smoothing linear operator that attenuetes the higher frequencies and leaves the low-frequency components in an image. Applying the edge detection algorithm on the Gaussian smoothed image allows us to get the high-level edges such as outlines and not the detailed edges.

The steps for edge detection with smoothing involve:
1. Convolving the Gaussian with the image to smooth the image
2. Applying the Sobel/Laplacian operator on the smoothed image to detect edges.

We can reduce the number of computations in the above steps with both the gradient operator(Sobel) and the Laplacian operators by understanding that the Gaussian operator and the gradient operators are linear. Hence, associatively, we can:
1. Apply the Sobel/Laplacian operator to the Gaussian.
2. Convolve the image with the new operator from 1 above; the **derivative of Gaussian** for Sobel and the **Laplacian of Gaussian** for Laplacian.

### An Example (Laplacian of Gaussian)

In [ ]:
# We are going to detect high-level edges from
# the flower above using the LoG operator.
#the gaussian low pass filter
import math
def distance(point1,point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def gaussianLP(D0,imgShape):
    base = np.zeros(imgShape[:2])
    rows, cols = imgShape[:2]
    center = (rows/2,cols/2)
    for x in range(cols):
        for y in range(rows):
            base[y,x] = math.exp(((-distance((y,x),center)**2)/(2*(D0**2))))
    return base

In [ ]:
#For Gaussian Smoothing
def convolve(image, filter_used):
    img_in_freq_domain = np.fft.fft2(image)

    # Shift the zero-frequency component to the center of the frequency spectrum
    centered = np.fft.fftshift(img_in_freq_domain)

    # Multiply the filter with the centered spectrum
    filtered_image_in_freq_domain = centered * np.fft.fftshift(np.fft.fft2(filter_used))

    # Shift the zero-frequency component back to the top-left corner of the frequency spectrum
    inverse_fftshift_on_filtered_image = np.fft.ifft2(filtered_image_in_freq_domain)

    # Apply the inverse Fourier transform to obtain the final filtered image
    final_filtered_image = np.fft.ifftshift(inverse_fftshift_on_filtered_image)

    return np.abs(final_filtered_image)


Above code was sourced from: [Source](https://medium.com/@mahmed31098/image-processing-with-python-frequency-domain-filtering-for-noise-reduction-and-image-enhancement-d917e449db68 ).

In [ ]:
#creating the Gaussian
sigma=10
gauss_flower = gaussianLP(sigma, icon.shape)
#We will now create a LoG for the Gaussian of the image
log_flower = cv2.Laplacian(gauss_flower, cv2.CV_64F, ksize=3)

plt.figure(figsize=[8, 8])
plt.subplot(121); plt.imshow(gauss_flower, cmap='gray'); plt.title(f'Gaussian, Sigma = {sigma}')
plt.subplot(122); plt.imshow(log_flower, cmap='gray'); plt.title('Laplacian of Gaussian')

Now we have the Laplacian of Gaussian for the image. We can then convolve it with the image. It is important to note that the parameter `D0` controls the width of the Gaussian. Increasing the diameter increases the number of frequencies that pass, hence the number of edges is increased. We will see the effect of this change in a few.

In [ ]:
# We will now convolve the LoG with the image
# The `filter2D` correlates but since the gaussian is
# reflected along the diagonal, this should be no
# problem.

# Getting the edge map
log_flower_edges = convolve(icon, log_flower)

#Thresholding to choose the zero-crossing for the edges
_, log_flower_zeros = cv2.threshold(log_flower_edges, 280, np.max(log_flower_edges), cv2.THRESH_BINARY)

#displaying
plt.figure(figsize=[7, 8])
plt.subplot(121); plt.imshow(log_flower_edges, cmap='gray'); plt.title('Edge Map')
plt.subplot(122); plt.imshow(log_flower_zeros, cmap='gray'); plt.title('Edges')

In [ ]:
np.max(log_flower_edges)

Here, we can see that we only captured the high level and outer edges of the flower as opposed to the first image where we captured even the interior details.

### Effect of Sigma on number of edges captured

We will briefly see the effect of Sigma on the number of image edges found. We will vary the threshold value for each image to find the zero-crossing that works best for each edge map.

In [ ]:
# A function that creates a LoG operator, convolves with
# image and produces an output of the threshold image
def logedges(sigma, threshold_value):
    lpgaussian = gaussianLP(sigma, icon.shape[:2]) #low-pass gaussian
    logoperator = cv2.Laplacian(lpgaussian, cv2.CV_64F, ksize=3) #LoG operator
    
    #convolving
    edgemap = cv2.filter2D(icon, cv2.CV_64F, logoperator)
    #thresholding
    _, loc_edges = cv2.threshold(edgemap, threshold_value, np.max(edgemap), cv2.THRESH_BINARY)

    return edgemap

In [ ]:
edges_1 = logedges(1, 35)
edges_5 = logedges(5, 70)
edges_10 = logedges(10, 135)

In [ ]:
plt.figure(figsize=[15, 10])
plt.subplot(131); plt.imshow(edges_1, cmap='gray'); plt.title('Sigma 1')
plt.subplot(132); plt.imshow(edges_5, cmap='gray'); plt.title('Sigma 5')
plt.subplot(133); plt.imshow(edges_10, cmap='gray'); plt.title('Sigma 10')

We can see that as the sigma level increases we can only capture high-level edges. We see that using sigma = 1 captures as edges much of the flower, whereas sigma = 5 captures less of the interior details but more of the exterior.

## Canny Edge Detection

The canny edge detection algorithm is one of the most widely used edge-detection algorithms. It uses aspects of the Sobel and Laplacian operators.

It follows the following steps:
1. Gaussian smoothing to remove noise.
2. Find the intensity gradient at each pixel.
3. Non-maximum suppression - for every pixel, check whether it is the maximum in its local neighborhood, suppress if not.
4. Double thresholding - to segment strong, weak and no edges.
5. Hysteresis thresholding to determine if the weak edges are connected to strong edges and and hence to count or, if not, make them irrelevant.

In [ ]:
#loading the image
face = cv2.imread('images/face_2.jpg', cv2.IMREAD_GRAYSCALE)
plt.imshow(face, cmap='gray')

### Getting the intensity gradient

We are going to begin by getting the image intensity gradient to calculate the magnitude and orientation of the image. The method `derivative_gaussian` will return the magnitude and the gradient orientation of the image.

In [ ]:
# getting the derivative of Gaussian
# the gradient magnitude
# the gradient orientation

def derivative_gaussian(img, sigma=1):
    #smoothing
    face_gaussian = convolve(img, gaussianLP(sigma, img.shape))
    
    #getting the x and y Sobel gradients
    Ix = cv2.Sobel(face_gaussian, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(face_gaussian, cv2.CV_64F, 0, 1, ksize=3)

    magnitude = np.hypot(Ix, Iy)
    magnitude = magnitude/magnitude.max() * 255
    direction = np.arctan2(Ix, Iy)

    return magnitude, direction

In [ ]:
edgemap_other, direction_map = derivative_gaussian(face, 2.2)
plt.imshow(edgemap_other, cmap='gray')
_ = plt.title('Gradient Magnitude')

### Non-maximum suppression

Ideally, the edges must be thin. Hence we will apply non-maximum suppression to the gradient magnitude to suppress values that are not maximum in their local neighborhood.

Sources: [1](https://towardsdatascience.com/canny-edge-detection-step-by-step-in-python-computer-vision-b49c3a2d8123), [2](https://en.wikipedia.org/wiki/Canny_edge_detector)

In [ ]:
# We will go through the image, at every point of grad.
# we will find the gradient

def edge_thinning_one(edgemap, orientmap):
    final = np.zeros(edgemap.shape, np.int32)
    for x in range(1, edgemap.shape[0] - 1):
        for y in range(1, edgemap.shape[1] - 1):
            p = 255
            q = 255
            #Get the angle
            theta = np.abs(np.rad2deg(orientmap[x, y]))
            if (0 <= theta < 22.5) or (157.5 <= theta <= 180):
                # 0, 180
                p = edgemap[x, y-1]
                q = edgemap[x, y+1]
            elif (22.5 <= theta < 67.5):
                # 45
                p = edgemap[x+1, y-1]
                q = edgemap[x-1, y+1]
            elif (67.5 <= theta < 112.5):
                # 90
                p = edgemap[x-1, y]
                q = edgemap[x+1, y]
            elif (112.5 <= theta < 157.5):
                # 135
                p = edgemap[x-1, y-1]
                q = edgemap[x+1, y+1]
            else:
                # For errors
                pass

            #check if the magnitude at point is greater than p,q
            if (edgemap[x, y] >= p) and (edgemap[x, y] >= q):
                final[x, y] = edgemap[x, y]
            else:
                final[x, y] = 0
    return final

In [ ]:
thin_edges = edge_thinning_one(edgemap_other, direction_map)
plt.figure(figsize=[7, 7])
plt.imshow(thin_edges, cmap='gray')

### Double thresholding

We will now apply a max and min threshold to further segment our image. Values above the maximum threshold will be considered strong edges while values below will be considered non-relevant. Values between will be considered weak edges and will only be elevated in the next step.

In [ ]:
def double_threshold(edgearray, lowbound = 40, highbound = 80):
    final_edge_array = np.zeros(edgearray.shape, np.uint8)
    
    #finding the edges
    #strong edges
    strong_edges_x, strong_edges_y = np.where(edgearray >= highbound)
    final_edge_array[strong_edges_x, strong_edges_y] = 255

    #weak edges
    weak_edges_x, weak_edges_y = np.where((edgearray >= lowbound) & (edgearray < highbound))
    final_edge_array[weak_edges_x, weak_edges_y] = highbound - ((highbound-lowbound)//2)
    
    return final_edge_array

In [ ]:
threshold_image = double_threshold(thin_edges, 35, 50)
plt.figure(figsize=[10, 10])
plt.imshow(threshold_image, cmap='gray')

### Edge tracking by hysteresis

We iterate through the weak edges to see if there are any strong edges around their 8-pixel neighborhood. IF there are, we call it a strong edge, otherwise it will not make the cut.

In [ ]:
def hysteresis_thresh(edge_image):
    final_image = edge_image.copy()

    #local neighborhood
    row = np.array([-1, -1, -1, 0, 0, 0, 1, 1, 1])
    col = np.array([-1, 0, 1, -1, 0, 1, -1, 0, 1])
    
    for x in range(1, edge_image.shape[0] - 1):
        for y in range(1, edge_image.shape[1] - 1):
            #only the weak edges
            #if (edge_image[x, y] > 0) and (edge_image[x, y] < 255):
            max = 0
            for pos in range(len(row)):
                a = x + row[pos]
                b = y + col[pos]
                
                if edge_image[a, b] == 255:
                    max = edge_image[a, b]
        
            final_image[x, y] = max

    return final_image

In [ ]:
final_piece = hysteresis_thresh(threshold_image)
plt.figure(figsize=[10, 10])
plt.imshow(final_piece, cmap='gray')

In [ ]:
np.unique(final_piece)

In [ ]:
np.unique(threshold_image)

In [ ]:
np.unique(thin_edges)

In [ ]:
plt.imshow(cv2.Canny(face, 100, 200))